# Sistemas de Recomendación - Netflix Prize Challenge

## Filtro Colaborativo a partir de Similitud Coseno

Se busca encontrar similitudes entre distancias Coseno

En este notebook vamos a implementar un sistema de recomendación a través de un filtro colaborativo a partir de la similitud coseno. Nos basamos en este [muy recomendable artículo](https://medium.com/radon-dev/item-item-collaborative-filtering-with-binary-or-unary-data-e8f0b465b2c3) para esta sección.

Comenzamos cargando los datos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector

In [ ]:
df1 = pd.read_csv('combined_data_1_with_movie_id.csv', dtype={'Rating': np.int8, 'movie_id': np.int16})
print(df1.shape)
df1.head()

(24053764, 3)


,User,Rating,Movie_id
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


Y los títulos.

In [ ]:
df_title = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])
df_title.head()

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


Vamos a descartar usuarios con el objetivo de achicar la base de datos. Bien hecho, no debería afectar al sistema de recomendación de manera apreciable. Nosotros lo vamos a hacer de una manera *ad-hoc*, pero recalcamos que existen criterios (que no estamos mencionando) para hacerlo de una manera racional.

Vamos a descartar aquellos usuarios que califican poco (menos de cinco calificaciones) o mucho (más de 100 calificaciones).

In [ ]:
mask_usuarios_descartables = np.logical_or(df1.User.value_counts() <= 5, df1.User.value_counts() > 100)
usuarios_descartables = mask_usuarios_descartables[mask_usuarios_descartables].index.values
print(len(usuarios_descartables))

150301


In [ ]:
mascara_descartables = df1.User.isin(usuarios_descartables)
print(mascara_descartables.sum())

13395316


In [ ]:
print(df1.shape)
df1 = df1[~mascara_descartables]
print(df1.shape)

(24053764, 3)
(10658448, 3)


También vamos a descartar también aquellas películas que tengan pocas calificaciones. Esto, nuevamente, no es obligatorio y lo hacemos con el objetivo de achicar la matriz de utilidad aún más.

In [ ]:
mask_items_descartables = df1.Movie_id.value_counts() <= 3000
# mask_items_descartables
items_descartables = mask_items_descartables[mask_items_descartables].index.values
# items_descartables
print(len(items_descartables))

3997


In [ ]:
mascara_descartables = df1.Movie_id.isin(items_descartables)
print(mascara_descartables.sum())

1606917


In [ ]:
print(df1.shape)
df1 = df1[~mascara_descartables]
print(df1.shape)

(10658448, 3)
(9051531, 3)


Notar que `df1` **no** es una matriz de utilidad, sino que son simplemente entradas con cada calificación. Esto es conveniente porque ocupa **mucho menos** espacio en memoria. Pero, para poder continuar, es necesario llevarlo a una matriz de utilidad.

Sin embargo, hay que tener cuidado. Tenemos del orden de ~1000 ítems y ~500 mil usuarios, lo que da una matriz de 500.000.000 entradas. Pero la mayoría de estas entradas son 0, ya que solo contamos con aproxidamente 10.000.000 calificaciones (estos valores se modifican según cómo filtremos el dataset). Es decir, aproximadamente una de cada 50 posibles entradas. Si guardásemos esta información en un dataset común, casi seguro llenaríamos nuestra memoria RAM. Por suerte, existen los tipos de datos "ralos" (*sparse*), que hacen esto de manera eficiente.

Puedes leer más [aquí](https://pandas.pydata.org/docs/user_guide/sparse.html).

In [ ]:
usuarios = df1.User.unique()
items = df1.Movie_id.unique()

n_usuarios = len(usuarios)
n_items = len(items)
print(n_usuarios, n_items)

320181 502


Creamos una matriz rala de ceros, de `n_usuarios` filas y `n_items` columnas.

In [ ]:
matriz_utilidad = pd.DataFrame(np.zeros((n_usuarios, n_items)), index =  usuarios, columns = items).astype(pd.SparseDtype("float"))
matriz_utilidad.head()

,8,17,28,30,33,58,83,97,108,111,...,4402,4418,4420,4432,4472,4479,4488,4490,4492,4496
824097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2630686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Hasta ahora, la matriz no contiene las califiaciones, solo ceros, así que debemos llenarla. Al tratarse una matriz rala o esparza, debemos hacerlo de una manera particular. Puedes leer más [aquí](https://stackoverflow.com/questions/49032856/assign-values-to-sparsearray-in-pandas).

Llenemos la matriz con las calificaciones:

In [ ]:
### Iteramos sobre cada columna
for column in matriz_utilidad:
    # Save the original sparse format for reuse later
    spdtypes = matriz_utilidad.dtypes[column]

    # Convert concerned Series to dense format
    matriz_utilidad[column] = matriz_utilidad[column].sparse.to_dense()
    
    ### DEBEMOS LLENAR LA COLUMNA CON LAS CALIFICACIONES CORRESPONDIENTES
    mask_calificaciones_columna = df1.Movie_id == column
    calificaciones_columna = df1[mask_calificaciones_columna]

    matriz_utilidad[column].loc[calificaciones_columna.User.values] = calificaciones_columna.Rating.values

    # Back to the original sparse format
    matriz_utilidad[column] = matriz_utilidad[column].astype(spdtypes)

/home/esteban/miniconda3/envs/datascience/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
matriz_utilidad.head()

,8,17,28,30,33,58,83,97,108,111,...,4402,4418,4420,4432,4472,4479,4488,4490,4492,4496
824097,2.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0
2630686,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A cada columna le ponemos el nombre de la película correspondiente.

In [ ]:
matriz_utilidad.columns = df_title.loc[matriz_utilidad.columns]['Name'].values

In [ ]:
matriz_utilidad.head()

,What the #$*! Do We Know!?,7 Seconds,Lilo and Stitch,Something's Gotta Give,Aqua Teen Hunger Force: Vol. 1,Dragonheart,Silkwood,Mostly Martha,Spartan,Duplex (Widescreen),...,A Nightmare on Elm Street,Titan A.E.,Against the Ropes,The Italian Job,Love Actually,Ordinary People,Wonder Boys,Ned Kelly,Club Dread,Farewell My Concubine
824097,2.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0
2630686,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ya contamos con una matriz de utilidad. Ahora debemos implementar el método.

**Normalización**

Para implementar el filtro colaborativo, vamos a normalizar las calificaciones que ponen los usuarios. En palabras del artículo, 

>*Ésta es la idea de normalizar los vectores de usuario para que un usuario con muchas calificaciones contribuya menos a cualquier calificación individual. Esto quiere decir que un me gusta de un usuario al que solo le han gustado 10 artículos es más valioso para nosotros que un me gusta de alguien a quien le gusta todo lo que encuentra.*

Entonces, dadas las calificaciones que puso cada usuario/a, por ejemplo 5, 3, 4, 1, 2, etc., se calcula la magnitud de ese vector, que para el ejemplo es $\sqrt{5^2 + 3^2 + 4^2 + 1^2 + 2^2 + ...}$. Luego, se divide cada calificación por ese valor.

Este paso es el más sensible, desde el punto de vista de memoria RAM. Por lo que seguiremos usando matrices ralas, en este caso de Scipy. Asegúrate de que entiendes bien la celda.

In [ ]:
from scipy import sparse
data_sparse = sparse.csr_matrix(matriz_utilidad)

magnitude = np.sqrt(data_sparse.power(2).sum(axis=1))
matriz_utilidad = matriz_utilidad.divide(magnitude, axis='index')
matriz_utilidad

,What the #$*! Do We Know!?,7 Seconds,Lilo and Stitch,Something's Gotta Give,Aqua Teen Hunger Force: Vol. 1,Dragonheart,Silkwood,Mostly Martha,Spartan,Duplex (Widescreen),...,A Nightmare on Elm Street,Titan A.E.,Against the Ropes,The Italian Job,Love Actually,Ordinary People,Wonder Boys,Ned Kelly,Club Dread,Farewell My Concubine
824097,0.069048,0.0,0.0,0.172619,0.0,0.0,0.138095,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.138095,0.0,0.0,0.034524,0.0,0.0
2630686,0.201784,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
644003,0.133235,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.222058,0.000000,0.0,0.0,0.000000,0.0,0.0
229220,0.239904,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.239904,0.000000,0.0,0.0,0.000000,0.0,0.0
2580478,0.529813,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708048,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0
175653,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0
1602517,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0
2478092,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.0


Ya contamos con la matriz de utilidad normalizada. Ahora, debemos crear la matriz de similitudes. Es decir, utilizando la similitud coseno creamos una matriz donde se computa la similitud entre cada ítem. En este caso, las películas.

Recuerda que dos ítems son similares si fueron calificados de la misma forma por un conjunto de usuarios que coinciden en sus gustos.

**Lee la documentación de `cosine_similarity` en Scikit-Learn y asegúrate de entender la celda a continuación**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    
    data_sparse = sparse.csr_matrix(data_items) # Esto solo convierte el dataframe en una matriz rala de scipy.
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(matriz_utilidad)
data_matrix

,What the #$*! Do We Know!?,7 Seconds,Lilo and Stitch,Something's Gotta Give,Aqua Teen Hunger Force: Vol. 1,Dragonheart,Silkwood,Mostly Martha,Spartan,Duplex (Widescreen),...,A Nightmare on Elm Street,Titan A.E.,Against the Ropes,The Italian Job,Love Actually,Ordinary People,Wonder Boys,Ned Kelly,Club Dread,Farewell My Concubine
What the #$*! Do We Know!?,1.000000,0.011905,0.015454,0.054690,0.016012,0.012753,0.022939,0.036692,0.027460,0.017864,...,0.017267,0.006643,0.015534,0.055275,0.063326,0.030412,0.033336,0.024705,0.012083,0.022010
7 Seconds,0.011905,1.000000,0.013997,0.040711,0.003074,0.011263,0.002889,0.002340,0.056138,0.027385,...,0.011350,0.007316,0.042302,0.058439,0.022166,0.001899,0.001996,0.028486,0.017042,0.001223
Lilo and Stitch,0.015454,0.013997,1.000000,0.097116,0.015679,0.046203,0.024125,0.020717,0.020254,0.048847,...,0.039026,0.066965,0.027682,0.100865,0.075114,0.025148,0.031075,0.016239,0.015367,0.014545
Something's Gotta Give,0.054690,0.040711,0.097116,1.000000,0.013321,0.035943,0.065419,0.072694,0.108205,0.166989,...,0.048037,0.025697,0.116965,0.291946,0.336159,0.072180,0.078586,0.071012,0.045897,0.032856
Aqua Teen Hunger Force: Vol. 1,0.016012,0.003074,0.015679,0.013321,1.000000,0.009890,0.001705,0.002561,0.006915,0.010271,...,0.033177,0.010851,0.001245,0.034127,0.019776,0.006121,0.016666,0.003570,0.025059,0.003930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ordinary People,0.030412,0.001899,0.025148,0.072180,0.006121,0.007013,0.155419,0.040992,0.011486,0.011628,...,0.032110,0.007670,0.009228,0.050351,0.064923,1.000000,0.099439,0.010808,0.002257,0.043569
Wonder Boys,0.033336,0.001996,0.031075,0.078586,0.016666,0.006976,0.062852,0.049669,0.027100,0.024506,...,0.034292,0.019061,0.015175,0.073341,0.094150,0.099439,1.000000,0.015368,0.011939,0.038016
Ned Kelly,0.024705,0.028486,0.016239,0.071012,0.003570,0.010441,0.008071,0.023551,0.100836,0.054554,...,0.010902,0.007812,0.085792,0.063806,0.071381,0.010808,0.015368,1.000000,0.024072,0.008339
Club Dread,0.012083,0.017042,0.015367,0.045897,0.025059,0.011150,0.002182,0.003458,0.073032,0.060801,...,0.023658,0.008795,0.046724,0.053121,0.043658,0.002257,0.011939,0.024072,1.000000,0.001931


In [ ]:
data_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 502 entries, What the #$*! Do We Know!? to Farewell My Concubine
Columns: 502 entries, What the #$*! Do We Know!? to Farewell My Concubine
dtypes: float64(502)
memory usage: 1.9+ MB


Podemos elegir un elemento de la matriz y ver los `nlargest` elementos más parecidos.

In [ ]:
print(data_matrix.loc["Men in Black II"].nlargest(25))

Men in Black II                                           1.000000
What Women Want                                           0.377085
Collateral Damage                                         0.350282
S.W.A.T.                                                  0.347777
Lethal Weapon 3                                           0.321729
Lethal Weapon                                             0.316907
Ghost                                                     0.309382
Pirates of the Caribbean: The Curse of the Black Pearl    0.276857
The Wedding Planner                                       0.270643
The Recruit                                               0.269769
The Sum of All Fears                                      0.260525
Hook                                                      0.258874
Bruce Almighty                                            0.257587
Braveheart                                                0.257529
The Matrix: Reloaded                                      0.25

Prueba con varios elementos. Elige películas conocidas y no tan conocidas. Algunas de directores famosos, *de autor* y franquicias. ¿Cómo te parece que funciona?

**Para pensar:** ¿Qué ocurre cuando vemos los parecidos de películas muy populares?¿Por qué será?¿Se te ocurre cómo arreglarlo?

Ahora vamos a construir el "vecindario" de cada ítem, que será un dataframe con los `n_neighbours` más similares. 


In [ ]:
n_neighbours = 20
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(n_neighbours))
for column in data_matrix.columns:
    data_neighbours.loc[column,:] = data_matrix.loc[:,column].sort_values(ascending=False)[:n_neighbours].index
data_neighbours

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
What the #$*! Do We Know!?,What the #$*! Do We Know!?,Sideways,Eternal Sunshine of the Spotless Mind,The Motorcycle Diaries,Finding Neverland,Super Size Me,Garden State,Ray,The Life Aquatic with Steve Zissou,Napoleon Dynamite,Bowling for Columbine,Being John Malkovich,Whale Rider,American Beauty,Frida,Kill Bill: Vol. 2,Outfoxed: Rupert Murdoch's War on Journalism,Rabbit-Proof Fence,Control Room,House of Sand and Fog
7 Seconds,7 Seconds,Hostage,King's Ransom,Coach Carter,Blade: Trinity,The Pacifier,Beauty Shop,Taxi,The Final Cut,Taking Lives,In Good Company,Batman Begins,S.W.A.T.,Man on Fire,The Amityville Horror,The Sandlot 2,The Bourne Supremacy,Unleashed,Saw,Ray
Lilo and Stitch,Lilo and Stitch,The Emperor's New Groove,Finding Nemo (Widescreen),Brother Bear (Theatrical Widescreen Version),Stuart Little 2,Aladdin: Platinum Edition,The Lion King: Special Edition,Shrek 2,Hercules,Sleeping Beauty: Special Edition,Spy Kids 2: The Island of Lost Dreams,Charlotte's Web,Pirates of the Caribbean: The Curse of the Bla...,Spy Kids,Lord of the Rings: The Fellowship of the Ring,Freaky Friday,The Wizard of Oz: Collector's Edition,Pocahontas,Jimmy Neutron: Boy Genius,Dinosaur
Something's Gotta Give,Something's Gotta Give,Love Actually,50 First Dates,Along Came Polly,The Last Samurai,Pirates of the Caribbean: The Curse of the Bla...,Secondhand Lions,The Italian Job,Man on Fire,Bruce Almighty,The Bourne Supremacy,Shrek 2,Road to Perdition,House of Sand and Fog,About Schmidt,Bend It Like Beckham,Ray,Mean Girls,Finding Nemo (Widescreen),Freaky Friday
Aqua Teen Hunger Force: Vol. 1,Aqua Teen Hunger Force: Vol. 1,Futurama: Vol. 1,Chappelle's Show: Season 1,Family Guy: Freakin' Sweet Collection,The Simpsons: Season 3,The Simpsons: Season 1,Cowboy Bebop Remix,The Simpsons: Treehouse of Horror,Half Baked,Clerks,Fear and Loathing in Las Vegas,Reservoir Dogs,Ghost in the Shell,Army of Darkness,Dogma,BASEketball,Napoleon Dynamite,Kill Bill: Vol. 2,The Life Aquatic with Steve Zissou,Being John Malkovich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ordinary People,Ordinary People,On Golden Pond,Moonstruck,Silkwood,Network,Midnight Cowboy,The Sting,American Beauty,A Beautiful Mind,Children of a Lesser God,The Deer Hunter,A Streetcar Named Desire,The Full Monty,The Way We Were,The Silence of the Lambs,Fried Green Tomatoes,Nine to Five,When Harry Met Sally,An Officer and a Gentleman,The Prince of Tides
Wonder Boys,Wonder Boys,High Fidelity,American Beauty,Being John Malkovich,The Ice Storm,About a Boy,About Schmidt,Igby Goes Down,This Is Spinal Tap,Clerks,Bowling for Columbine,Bottle Rocket,A Beautiful Mind,Reservoir Dogs,The Full Monty,Eternal Sunshine of the Spotless Mind,The Silence of the Lambs,Heathers,Moonstruck,Bend It Like Beckham
Ned Kelly,Ned Kelly,The Alamo,Taking Lives,Spartan,The Missing,Man on Fire,House of Sand and Fog,The Ladykillers,Against the Ropes,The Door in the Floor,The Last Samurai,Beyond Borders,The Bourne Supremacy,Secondhand Lions,The Magdalene Sisters,Kill Bill: Vol. 2,Along Came Polly,Love Actually,Something's Gotta Give,The Passion of the Christ
Club Dread,Club Dread,The Rundown,Harold and Kumar Go to White Castle,Dawn of the Dead,Kill Bill: Vol. 2,Along Came Polly,Spartan,American Wedding,National Lampoon's Van Wilder,The Last Samurai,Taking Lives,Mean Girls,50 First Dates,The Ladykillers,Freddy vs. Jason,Elf,Man on Fire,Duplex (Widescreen),Identity,BASEketball


Ahora que ya contamos con un una matriz de parecidos, es momento de hacer recomendaciones.

**Recomendaciones**

Elijamos un usuario al azar y veamos qué películas vio y sus calificaciones

In [ ]:
# np.random.seed(15)
user = np.random.choice(list(matriz_utilidad.index))
# user = 1329595  # Descomentar este usuario y ver sus resultados.
# user = 1208602  # Descomentar este usuario y ver sus resultados.
print(user)
df_user = df1[df1.User == user]
df_user['Title'] = df_title.loc[df1[df1.User == user].Movie_id]['Name'].values
df_user

1208602


<ipython-input-22-306e07f5e1d6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['Title'] = df_title.loc[df1[df1.User == user].Movie_id]['Name'].values


,User,Rating,Movie_id,Title
5163381,1208602,3,1046,Uptown Girls
5531474,1208602,1,1110,Secondhand Lions
5975399,1208602,2,1176,The Devil's Backbone
6706488,1208602,4,1307,S.W.A.T.
7164642,1208602,4,1428,The Recruit
7381280,1208602,4,1470,Bend It Like Beckham
7455544,1208602,1,1482,Beyond Borders
7975425,1208602,4,1595,The Third Man
9030339,1208602,4,1798,Lethal Weapon
14202996,1208602,3,2743,The Pianist


Podemos usar todas las películas que vio o filtrar solamente las que calificó sobre cierto umbral. **¿Te parece que mejoran las recomendaciones?**

In [ ]:
known_user_likes = df_user['Title'].values
# known_user_likes = df_user[COMPLETAR]['Title'].values # puedes resolver aqui el ejercicio.
known_user_likes

array(['28 Days Later'], dtype=object)

Una vez que ya sabemos qué le gustó, hay que tomar del vecindario los parecidos a cada película.

In [ ]:
most_similar_to_likes = data_neighbours.loc[known_user_likes]
most_similar_to_likes

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
28 Days Later,28 Days Later,Identity,Dawn of the Dead,The Matrix: Reloaded,Kill Bill: Vol. 2,X2: X-Men United,Pirates of the Caribbean: The Curse of the Bla...,Reservoir Dogs,Ghost Ship,Lord of the Rings: The Fellowship of the Ring,Evil Dead 2: Dead by Dawn,The Silence of the Lambs,Army of Darkness,Freddy vs. Jason,American Beauty,The Italian Job,Bowling for Columbine,A Nightmare on Elm Street,Eternal Sunshine of the Spotless Mind,Jaws


Hasta ahora tenemos un dataframe con todos los parecidos a las películas que vio. Notar que pueden haber películas repetidas, ya que una misma película puede parecerse a más de una que vio. Entonces creamos una lista con elementos sin repetir.

In [ ]:
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
similar_list

['Kill Bill: Vol. 2',
 'American Beauty',
 'The Matrix: Reloaded',
 'Reservoir Dogs',
 'Dawn of the Dead',
 'Army of Darkness',
 'A Nightmare on Elm Street',
 'Lord of the Rings: The Fellowship of the Ring',
 'Identity',
 '28 Days Later',
 'Jaws',
 'Evil Dead 2: Dead by Dawn',
 'X2: X-Men United',
 'Ghost Ship',
 'Bowling for Columbine',
 'Eternal Sunshine of the Spotless Mind',
 'The Italian Job',
 'Freddy vs. Jason',
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 'The Silence of the Lambs']

A partir de esta lista, tomamos de `neighbourhood` los elementos similares junto con la medida de similitud.

In [ ]:
neighbourhood = data_matrix[similar_list].loc[similar_list]
neighbourhood

,Kill Bill: Vol. 2,American Beauty,The Matrix: Reloaded,Reservoir Dogs,Dawn of the Dead,Army of Darkness,A Nightmare on Elm Street,Lord of the Rings: The Fellowship of the Ring,Identity,28 Days Later,Jaws,Evil Dead 2: Dead by Dawn,X2: X-Men United,Ghost Ship,Bowling for Columbine,Eternal Sunshine of the Spotless Mind,The Italian Job,Freddy vs. Jason,Pirates of the Caribbean: The Curse of the Black Pearl,The Silence of the Lambs
Kill Bill: Vol. 2,1.000000,0.257928,0.232347,0.296330,0.156837,0.117630,0.093935,0.293217,0.215883,0.172314,0.138313,0.093721,0.249867,0.104774,0.209367,0.288650,0.277850,0.084061,0.312045,0.245849
American Beauty,0.257928,1.000000,0.205033,0.345925,0.069954,0.105220,0.130635,0.287803,0.167041,0.146397,0.257623,0.087985,0.179911,0.108303,0.298267,0.296600,0.239076,0.045938,0.277418,0.357668
The Matrix: Reloaded,0.232347,0.205033,1.000000,0.182043,0.095074,0.097182,0.128835,0.343417,0.217410,0.173151,0.159884,0.073349,0.350244,0.171180,0.168267,0.150144,0.276605,0.104773,0.314812,0.207013
Reservoir Dogs,0.296330,0.345925,0.182043,1.000000,0.084429,0.153972,0.144207,0.246133,0.147887,0.160513,0.199862,0.128394,0.171916,0.075758,0.222762,0.237568,0.192573,0.050028,0.224079,0.310014
Dawn of the Dead,0.156837,0.069954,0.095074,0.084429,1.000000,0.097433,0.079924,0.098462,0.122949,0.179375,0.069186,0.099722,0.112967,0.113993,0.060071,0.099222,0.098258,0.138206,0.111154,0.088040
Army of Darkness,0.117630,0.105220,0.097182,0.153972,0.097433,1.000000,0.137268,0.119748,0.077028,0.151204,0.120362,0.415158,0.103780,0.063570,0.070560,0.084135,0.065434,0.069624,0.104099,0.120938
A Nightmare on Elm Street,0.093935,0.130635,0.128835,0.144207,0.079924,0.137268,1.000000,0.117295,0.096531,0.138506,0.204934,0.167869,0.087718,0.165320,0.080337,0.067573,0.089893,0.123244,0.119043,0.162129
Lord of the Rings: The Fellowship of the Ring,0.293217,0.287803,0.343417,0.246133,0.098462,0.119748,0.117295,1.000000,0.225935,0.156631,0.207792,0.087119,0.359660,0.141645,0.211889,0.230512,0.319996,0.074394,0.453016,0.311128
Identity,0.215883,0.167041,0.217410,0.147887,0.122949,0.077028,0.096531,0.225935,1.000000,0.203597,0.123558,0.064397,0.209690,0.184890,0.150892,0.166937,0.263242,0.127442,0.258396,0.195217
28 Days Later,0.172314,0.146397,0.173151,0.160513,0.179375,0.151204,0.138506,0.156631,0.203597,1.000000,0.134257,0.153545,0.169144,0.156893,0.138658,0.134932,0.140967,0.150781,0.161905,0.153244


Esta es, tal vez, la parte más abstracta para seguir. Ya sabemos cuáles elementos se parecen y tenemos medidas de similitud entre esos elementos, pero debemos condensar esta información en un único ranking. Para ello, tomamos de la matriz de utilidad las calificaciones de las películas que vio (que están normalizados) y las de las películas que no vio. Como no las vio, son cero.

In [ ]:
user_vector = matriz_utilidad.loc[user].loc[similar_list]
user_vector

Kill Bill: Vol. 2                                         0.000000
American Beauty                                           0.000000
The Matrix: Reloaded                                      0.000000
Reservoir Dogs                                            0.000000
Dawn of the Dead                                          0.000000
Army of Darkness                                          0.000000
A Nightmare on Elm Street                                 0.000000
Lord of the Rings: The Fellowship of the Ring             0.000000
Identity                                                  0.000000
28 Days Later                                             0.372678
Jaws                                                      0.000000
Evil Dead 2: Dead by Dawn                                 0.000000
X2: X-Men United                                          0.000000
Ghost Ship                                                0.000000
Bowling for Columbine                                     0.00

Esta celda calcula el score. No te preocupes si no la entiendes, pero si conoces algo de algebra lineal, es el producto escalar de la matriz "vecindario" con las calificaciones.

In [ ]:
# Calculate the score.
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))
score

Kill Bill: Vol. 2                                         0.013266
American Beauty                                           0.011215
The Matrix: Reloaded                                      0.013875
Reservoir Dogs                                            0.013077
Dawn of the Dead                                          0.022468
Army of Darkness                                          0.017210
A Nightmare on Elm Street                                 0.015477
Lord of the Rings: The Fellowship of the Ring             0.011043
Identity                                                  0.017985
28 Days Later                                             0.093732
Jaws                                                      0.012719
Evil Dead 2: Dead by Dawn                                 0.018639
X2: X-Men United                                          0.013747
Ghost Ship                                                0.017554
Bowling for Columbine                                     0.01

Sacamos de la lista las películas que ya vio.

In [ ]:
score = score.drop(known_user_likes)
score

Kill Bill: Vol. 2                                         0.013266
American Beauty                                           0.011215
The Matrix: Reloaded                                      0.013875
Reservoir Dogs                                            0.013077
Dawn of the Dead                                          0.022468
Army of Darkness                                          0.017210
A Nightmare on Elm Street                                 0.015477
Lord of the Rings: The Fellowship of the Ring             0.011043
Identity                                                  0.017985
Jaws                                                      0.012719
Evil Dead 2: Dead by Dawn                                 0.018639
X2: X-Men United                                          0.013747
Ghost Ship                                                0.017554
Bowling for Columbine                                     0.013236
Eternal Sunshine of the Spotless Mind                     0.01

E imprimimos las películas que le gustó y las recomendadas en orden.

In [ ]:
print(known_user_likes)
print(score.nlargest(20))

['28 Days Later']
Dawn of the Dead                                          0.022468
Freddy vs. Jason                                          0.020739
Evil Dead 2: Dead by Dawn                                 0.018639
Identity                                                  0.017985
Ghost Ship                                                0.017554
Army of Darkness                                          0.017210
A Nightmare on Elm Street                                 0.015477
The Matrix: Reloaded                                      0.013875
X2: X-Men United                                          0.013747
Kill Bill: Vol. 2                                         0.013266
Bowling for Columbine                                     0.013236
Reservoir Dogs                                            0.013077
Jaws                                                      0.012719
Eternal Sunshine of the Spotless Mind                     0.012205
The Silence of the Lambs                    

Si bien el notebook es demostrativo, te recomendamos que lo mires detenidamente, trates de entenderlo lo mejor posible y juegues un poco con él. 

**Para pensar:**

1. ¿Cuáles te parecen que son los hiperparámetros?
1. ¿Cómo evaluarías estos resultados?¿Cómo harías un train/test split?